# Download NDVI indices yearwise in Geotiff format

## Import packages

In [8]:
import geemap
import ee 
import os
import geemap.colormaps as cm
import warnings
warnings.filterwarnings('ignore')

## Create an ipyleaflet-based interactive map

In [5]:
# A Map object is created which helps to interact with a open street interactive map and add layers to it.
Map = geemap.Map(center=[40,100], zoom=4)

## Get the dataset from GEE server

In [6]:
# The landsat 7 data is called from the Google Earth Engine Server and a Image Collection is created.
dataset_l7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_RT_TOA')

# Run the code below to get more information about the dataset we want.
# print(dataset_l7.size().getInfo()) 

## Get the shapefile (locally in this case)

In [9]:
# Get the shapefile either locally or create a Geometry object for creating the Region of Interest of our project.
city_file = './shapefiles/Bengaluru-shapefile/BBMP_Boundary.shp'
roi = geemap.shp_to_ee(city_file)

# Add the roi (which is a FeatureCollection) to the interactive map
Map.addLayer(roi, {}, 'Bengaluru') 

In [137]:
# Bring the roi object to the center of the Map
Map.centerObject(roi)

* TODO
* find if any dataset available with all the shapefile
* prepare a function to get folderwise shapefiles if offline shapefiles are provided

## Helper Functions

In [10]:
def clip_to_shape(image):
    '''
    Function to take an image and clip it according to the region of interest(roi).
    It takes an image as an input and outputs the clipped version of the image according to roi
    '''before hand
    return image.clip(roi)

In [142]:
def maskclouds(image):
    '''
    Function to take an image and mask the clouds for better visibility. The returns the masked image.
    '''
    # Select the BQA band, which has the cloud information
    qa = image.select('BQA')

    # In BQA Band bit 4 is the cloud band, hence activate only that bit, set all other bits of BQA to zero
    cloudBit = 1 << 4 #(00000001 -> 00010000) 

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBit).eq(0) # And with all the other bits of BQA to check for clouds
    return image.updateMask(mask)

In [143]:
def ndvi_image_compute(img):
    '''
    Function to compute the ndvi index of the averaged image.It also initializes a download ready ndvi pallete for
    visualization and returns the final color image (with pallete)/.
    '''
    # Computes the nvdi index of the image which is (Band 4 – Band 3) / (Band 4 + Band 3) for LandSAT 7.
    ndvi_img = img.normalizedDifference(['B4', 'B3'])
    ndvi_visualization = {
#       'min': -0.22789797020331423,
#       'max': 0.3912415762453296,
#       'palette': 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400,' + \
#         '3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
      'palette': cm.palettes.ndvi,
        }
    # Get a corrected image with the pallete specified.
    corrected_img = ndvi_img.visualize(**ndvi_visualization)
    
    return corrected_img

In [144]:
def filter_date(year,i,flag):
    '''
    Function to compute the start and the end date of a particular season. It takes the year and the season of 
    which we need to find the date and returns the start and the end date is YYYY/MM/DD format
    '''
    start_date = ee.Date.fromYMD(year,seasons[i][1],seasons[i][0])
    if flag==0:
        end_date = ee.Date.fromYMD(year,seasons[i][3],seasons[i][2])
    else:
        end_date = ee.Date.fromYMD(year+1,seasons[i][3],seasons[i][2])
    
    return start_date, end_date
    

In [ ]:
def get_filtered_image(year,i,flag):
    '''
    Function to get the final computed image of the year specified ( sort by filterdate_ cloud_masked + mean +
    ndvi computed)/. The function returns the final computed image.
    '''
    start_date,end_date = filter_date(year,i,flag)
    
    img = (dataset_l7.filterDate(start_date,end_date)
                    .filterBounds(roi)
                    .map(maskclouds)
                    .select('B3','B4')
                    .mean())
    img = ndvi_image_compute(img)
    return img
    

In [ ]:
def yearlyMap(year):
    '''
    Function to loop over the five selected intervals over an year by taking a year input.
    '''
    year_folder = str(year)
    for k,season in enumerate(seasons):
        if season=='djf' or season== 'ann':
            img = get_filtered_image(year,season,flag=1) 
        else:
            img = get_filtered_image(year,season,flag=0)
        filename = season + '-' + str(year)
        download_to_drive(filename, year_folder,img)

In [145]:
def download_to_drive(filename,year_folder,img):
    '''
    Function to download the images year wise into the authenticated drive in a geotiff format in the specified
    year folders.
    '''
    task = ee.batch.Export.image.toDrive(image= img.clip(roi),
                                     description='ndvi index computation for two decades',
                                     scale=30,
                                     region=roi.geometry(),
                                     folder=year_folder,
                                     fileNamePrefix=filename,
                                     fileFormat='GeoTIFF')
    task.start()

## External Paramter Initialization

In [ ]:
# Python dictionary where the 'key' states the yearly timeframes to be evaluated
# mam: mar-apr-may; jjas: jun-jul-aug-sep; on: oct-nov; djf: dec-jan-feb; ann: annual
# The values indicates the stating and the ending dates of the timesframes to be evaluated
# [start date, start month, end date, end month]
seasons = {'mam':[1,3,31,5],'jjas':[1,6,30,9],'on':[1,10,30,11],'djf':[1,12,28,2],'ann':[1,3,28,2]}

# State the start and end year of evaluation
start_year = 2000 
end_year = 2002

In [ ]:
# List of years
years = [y for y in range(start_year,end_year)]

In [147]:
# Loop the computation over the specified list of years
for year in years:
    yearlyMap(year)